## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [10]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
# groq_api_key



In [13]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-2.5-32b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000200E8C024D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000200E8CE8650>, model_name='qwen-2.5-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Nick and I am a Chief AI Engineer")])

AIMessage(content="Hello Nick! It's great to meet you. As a Chief AI Engineer, you must be involved in some fascinating projects and challenges. How can I assist you today? Is there anything specific related to AI engineering, technology trends, or any other area where you need information or advice?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 42, 'total_tokens': 100, 'completion_time': 0.29, 'prompt_time': 0.004364282, 'queue_time': 0.211272235, 'total_time': 0.294364282}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-e15b88cc-3fdb-4afd-997d-45b44a35a236-0', usage_metadata={'input_tokens': 42, 'output_tokens': 58, 'total_tokens': 100})

In [15]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nick and I am a Chief AI Engineer"),
        AIMessage(content="Hello Nick! It's great to meet you. \nAs a Chief AI Engineer, you must be involved in some fascinating projects and challenges. \nHow can I assist you today? Is there anything specific related to AI engineering, technology trends, or any other area where you need information or advice?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is Nick, and you are a Chief AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 122, 'total_tokens': 136, 'completion_time': 0.07, 'prompt_time': 0.00846503, 'queue_time': 0.212746206, 'total_time': 0.07846503}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-0af4b50e-b8ea-4174-9795-fad025261d62-0', usage_metadata={'input_tokens': 122, 'output_tokens': 14, 'total_tokens': 136})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [16]:
!pip install langchain_community

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# This is a dictionary to store the message history for each session
store={}
# The key is the session ID, and the value is an instance of ChatMessageHistory
# this is a function to get the message history for a given session ID
# The function checks if the session ID exists in the store; if not, it creates a new ChatMessageHistory instance
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [18]:
config={"configurable":{"session_id":"chat1"}}

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nick and I am a Chief AI Engineer")],
    config=config
)

In [20]:
response.content

"Hello Nick! It's great to meet you. As a Chief AI Engineer, I'm sure you're involved in some fascinating projects. What kind of AI technologies are you currently working on or interested in exploring? If you have any specific questions or need insights on recent advancements in AI engineering, feel free to ask!"

In [21]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Nick.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 120, 'total_tokens': 126, 'completion_time': 0.03, 'prompt_time': 0.010500436, 'queue_time': 47.857533694, 'total_time': 0.040500436}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-f3b605cc-0ec0-4404-b151-32d4132960f6-0', usage_metadata={'input_tokens': 120, 'output_tokens': 6, 'total_tokens': 126})

In [22]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Hello! Since we just met, I don't actually know your name. Could you tell me your name? I'd love to know!"

In [23]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! How can I assist you today?'

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

### Prompt templates
* `Prompt Templates` help to `turn raw user information` into a `format that the LLM can work with`. 
* In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. 
* First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nick")]})

AIMessage(content="Hello Nick! It's nice to meet you. How can I assist you today? Is there anything specific you would like to know or discuss?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 34, 'total_tokens': 64, 'completion_time': 0.15, 'prompt_time': 0.00391543, 'queue_time': 0.087382191, 'total_time': 0.15391543}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-68c769d1-71e2-45e0-9826-e0a3fe84af12-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64})

In [ ]:
## This is a dictionary to store the message history for each session
## The RunnableWithMessageHistory will use this to keep track of the message history for each session
## The get_session_history function will check if the session ID exists in the store; if not, it creates a new ChatMessageHistory instance
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [27]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nick")],
    config=config
)

response

AIMessage(content="Hello Nick! It's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 34, 'total_tokens': 52, 'completion_time': 0.09, 'prompt_time': 0.0039009, 'queue_time': 0.177260289, 'total_time': 0.0939009}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_92412bc7e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-650dbc00-5b9c-469f-ad08-5e21281a3339-0', usage_metadata={'input_tokens': 34, 'output_tokens': 18, 'total_tokens': 52})

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Nick.'

In [ ]:
## Add more complexity
# Add a language parameter to the system message
# The system message will now include a placeholder for the language variable
# The language variable will be passed in when invoking the chain
# this function will be used to set the language variable in the system message
# The MessagesPlaceholder will be used to replace the {language} placeholder in the system message with the actual language value
# The language variable will be passed in when invoking the chain
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Nick")],"language":"French"})
response.content

"Bonjour Nick, comment puis-je vous aider aujourd'hui ?"

* Let's now wrap this more complicated chain in a Message History class. 
* This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [32]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [33]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Nick")],"language":"French"},
    config=config
)
repsonse.content

"Bonjour Nick, comment puis-je vous aider aujourd'hui ?"

In [34]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "French"},
    config=config,
)

In [35]:
response.content

'Ton nom est Nick.'

### Managing the Conversation History
* One important concept to understand when building chatbots is how to manage conversation history. 
* If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. 
* Therefore, it is important to add a step that limits the size of the messages you are passing in.
* `'trim_messages'` helper to `reduce how many messages` `we're sending` to the `model`. 
* The `trimmer` allows us to `specify how many tokens` we `want to keep`, along with other parameters like if we want to always keep the system message and whether to allow partial messages. 

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
## trim_messages is a utility function that helps to trim the messages in a conversation to a specified length
## The function takes a list of messages and trims them to the specified length based on the strategy provided
## The strategy can be "last" or "first", which determines whether to keep the last or first messages in the conversation
## The function also takes a token counter to count the number of tokens in the messages
## allow_partial is a boolean that determines whether to allow partial messages in the conversation
## The function also takes a start_on parameter to determine whether to start trimming from the human or AI message.
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\MyProjects\A.I-Projects\Langchain-for-GenAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\MyProjects\A.I-Projects\Langchain-for-GenAI\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nickb\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
## RunnablePassthrough is a class that allows you to pass through the input without any modifications
## It is useful when you want to create a chain of runnables and need a placeholder for a runnable that does nothing
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I don't have specific information about your preferences. However, some popular ice cream flavors include vanilla, chocolate, strawberry, mint chocolate chip, and cookies and cream. Could you tell me if there's a particular flavor you enjoy?"

In [38]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is. The answer is 4.'

In [ ]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [40]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to information about individual users unless you provide it. Can you tell me your name?"

In [41]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"It seems like there might have been a misunderstanding or a missing part of your message. You didn't provide a specific math problem in your question. Could you please provide more details or the specific math problem you're referring to?"